## 0. 前置知识
### (1)残差块。
结构上，让输入数据**跨层**更快地向前传播。
理论上，让每个附加层都应该更容易地**包含原始函数**作为其元素之一。
其优势是能拟合出更优的解，更容易降低训练误差。
### (2)批量规范化。
##### 待解决的问题
网络层数一多(形如 x → l1 → l2 → l3 → ... → ln → loss 函数)，规范化就不可避免。因为损失函数在最后层，反向传播时后面的层训练的比较快；而前面的层训练较慢。前面的层一变化，后面所有都得跟着变。导致后面的层重新学习了多次。最终导致收敛变慢。批量规范化，就是希望**前面层改变时，后面的层不必重新学**，以**加速收敛**速度，以前的学习率可能是0.01，现在可设lr=0.1。

##### 用在哪里
- 可学习的参数为 $\gamma$和$\beta$
- 作用在
    - 全连接层和卷积层的输出上，激活函数前
    - 全连接层和卷积层的输入上
- 对全连接层，作用在特征维
- 对于卷积层，作用在通道维

##### 具体在做什么
理论还不能解释，深度学习的工程是走在理论前面的。
- 批量归一化**固定**小批量中的**均值**和**方差**，然后学习出适合的偏移和缩放
- 可以加速收敛速度，但一般不改变模型精度

### (3)Embedding
将字词变成向量，两个近义词有更近的向量距离。
Embedding层，在某种程度上，就是用来降维的，降维的原理就是**矩阵乘法**。
假如我们有一个(10W,10W)的矩阵，用它乘上一个(10W,20)的矩阵，我们可以把它降到(10W,20)，瞬间量级降了10W/20=5000倍

## 1. 模型
采用的注意力评分函数是缩放点积评分。
### (1) 编码器
编码器是由多个相同的层叠加而成的，每个层都有两个子层 sublayer:
- 自注意力层
- 基于位置的前馈网络

编码器在计算自注意力时，查询、键和值都来自前一个编码器层的输出。
因为采用了残差连接，要求输入的$X$在流经网络时尺寸不得发生变化。即对于序列中任何位置的任何输入
$x \in \mathbb{R}^d$，都要求满足$sublayer(x) \in \mathbb{R}^d$，以便残差连接满足$$x + sublayer(x) \in \mathbb{R}^d$$。

在残差连接的加法计算之后，紧接着应用层规范化(layer normalization)

### (2) 解码器

第一个遮蔽多头注意力 sublayer 使用遮蔽矩阵，要求解码器中的每个位置只能考虑该位置之前的所有位置，确保预测仅依赖于已生成的输出词元。

In [5]:
import math
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

## 2. 基于位置的前馈网络
对序列中的所有位置的表示进行变换时使用的是同一个多层感知机（MLP）

输入 $x$: (batch_size,num_step,num_hidden)
通过一个两层的感知机: (num_hidden,num_hidden,num_outputs),转为
输出: (batch_size,num_step,num_outputs)

In [7]:
class PositionWiseFFN(nn.Module):
    """基于位置的前馈网络"""

    def __init__(self, ffn_num_input, ffn_num_hiddens, fnn_num_outputs, **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, fnn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))


举例：验证基于位置的前馈网络的数据流转

In [13]:
ffn = PositionWiseFFN(4, 4, 8)
ffn.eval()
# 传入 1 个 batch,每个 batch 有 2 个时间步（也可称为子序列），每个时间步长度=4=ffn_num_input
ffn(torch.ones(2, 3, 4)).shape


torch.Size([2, 3, 8])

## 3. 残差连接和层规范化（add & norm）
在NLP中（输入通常是变长序列）批量规范化通常不如层规范化的效果好。

In [14]:
ln = nn.LayerNorm(2)  # 每个 batch 单独进行归一化
bn = nn.BatchNorm1d(2)  # 所有 batch 一起进行归一化

X = torch.tensor([[1, 2], [2, 3]], dtype=torch.float32)
# 在训练模式下计算X的均值和方差
print('layer norm:', ln(X), '\nbatch norm:', bn(X))

layer norm: tensor([[-1.0000,  1.0000],
        [-1.0000,  1.0000]], grad_fn=<NativeLayerNormBackward0>) 
batch norm: tensor([[-1.0000, -1.0000],
        [ 1.0000,  1.0000]], grad_fn=<NativeBatchNormBackward0>)


使用残差连接和层规范化来实现AddNorm类

In [27]:
class AddNorm(nn.Module):
    """残差连接后进行层规范化"""

    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)

    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)  # dropout(Y) + X 即残差操作，ln()进行规范化

残差连接要求两个输入的形状相同，加法操作后输出张量的形状也一致。

In [28]:
add_norm = AddNorm([3, 4], 0.5)
add_norm.eval()
add_norm(torch.ones((2, 3, 4)), torch.ones((2, 3, 4))).shape

torch.Size([2, 3, 4])

## 4.编码器
编码器的两个 `sublayer` 已经定义完成，开始构造编码器的单层组件

In [32]:
class EncoderBlock(nn.Module):
    """Transformer 编码器块"""

    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, dropout,
                 use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = d2l.MultiHeadAttention(key_size, query_size, value_size, num_hiddens,
                                                num_heads, dropout, use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens, ffn_num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)

    def forward(self, X, valid_lens):
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        return self.addnorm2(Y, self.ffn(Y))

编码器中的任何层都不会改变其输入的形状。

In [33]:
X = torch.ones((2, 100, 24))
valid_lens = torch.tensor([3, 2])
encoder_blk = EncoderBlock(24, 24, 24, 24, [100, 24], 24, 48, 8, 0.5)
encoder_blk.eval()
encoder_blk(X, valid_lens).shape

torch.Size([2, 100, 24])

In [39]:
class TransformerEncoder(d2l.Encoder):
    """Transformer编码器"""

    def __init__(self, vocab_size, key_size, query_size, value_size,
                 num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens,
                 num_heads, num_layers, dropout, use_bias=False, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(vocab_size, num_hiddens)  # 每个嵌入向量的大小 = 隐藏层单元数
        self.pos_encoding = d2l.PositionalEncoding(num_hiddens, dropout)
        self.blocks = nn.Sequential()

        # 编码器第 i 层自注意力接受的查询、键和值都来自第 i-1 层的自注意力的输出
        for i in num_layers:
            self.blocks.add_module("block" + str(i),
                                   EncoderBlock(key_size, query_size, value_size, num_hiddens, norm_shape,
                                                ffn_num_input, ffn_num_hiddens, num_heads, dropout, use_bias))

    def forward(self, X, valid_lens, *args):
        # 因为位置编码值在-1和1之间，
        # 因此嵌入值乘以嵌入维度的平方根进行缩放，
        # 然后再与位置编码相加。
        X = self.pos_encoding(self.embedding(X) * math.sqrt(self.num_hiddens))
        self.attention_weights = [None] * len(self.blocks)
        for i, blk in enumerate(self.blocks):
            X = blk(X, valid_lens)
            self.attention_weights[i] = blk.attention.attention.attention_weights
        return X


下面我们指定了超参数来创建一个$2$层的Transformer编码器。
Transformer编码器输出的形状是（批量大小，时间步数目，num_hiddens）。

In [40]:
encoder = TransformerEncoder(
    200, 24, 24, 24, 24, [100, 24], 24, 48, 8, 2, 0.5)
encoder.eval()
encoder(torch.ones((2, 100), dtype=torch.long), valid_lens).shape

torch.Size([2, 100, 24])

关于 nn.Embedding()

In [36]:
# @param:   num_embeddings      嵌入词表的尺寸
# @param:   embedding_dim       要求每个嵌入向量的size，transformer 要求为 3
embedding = nn.Embedding(10, 3)

# a batch of 2 samples of 4 indices each
i = torch.LongTensor([[1, 2, 3, 4, 5], [2, 2, 3, 4, 5]])
embedding(i)

tensor([[[ 0.9862,  1.1006,  0.9881],
         [ 2.0816, -1.8696, -1.1936],
         [ 0.4373, -0.8299, -0.3389],
         [ 0.5716,  1.4253,  0.3239],
         [ 1.1779, -2.2216,  1.4105]],

        [[ 2.0816, -1.8696, -1.1936],
         [ 2.0816, -1.8696, -1.1936],
         [ 0.4373, -0.8299, -0.3389],
         [ 0.5716,  1.4253,  0.3239],
         [ 1.1779, -2.2216,  1.4105]]], grad_fn=<EmbeddingBackward0>)

## 5. 解码器
由多个相同的层组成。
在DecoderBlock 类中实现的每个层包含了三个子层：解码器自注意力、“编码器-解码器”注意力和基于位置的前馈网络。
这些子层也都被 Addnorm 层规范化围绕。

参数dec_valid_lens 确保任何查询都只会与解码器中所有**已经生成词元**的位置（即直到该查询位置为止）进行注意力计算。
> 关于序列到序列模型（sequence-to-sequence model），在训练阶段，其输出序列的所有位置（时间步）的词元都是已知的；然而，在预测阶段，其输出序列的词元是逐个生成的。因此，在任何解码器时间步中，**只有生成的词元才能用于解码器的自注意力计算中**。为了在解码器中保留自回归的属性，其掩蔽自注意力设定了参数dec_valid_lens，以便任何查询都只会与解码器中所有已经生成词元的位置（即直到该查询位置为止）进行注意力计算。

In [ ]:
class DecoderBlock(nn.Module):
    """解码器中第i个块"""

    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, i, **kwargs):
        super(DecoderBlock, self).__init__(**kwargs)
        self.i = i  # 标识当前第 i 个 block
        self.attention1 = d2l.MultiHeadAttention(key_size, query_size, value_size, num_hiddens, num_heads, dropout)
        self.addnorm1 = AddNorm(norm_shape, dropout)

        self.attention2 = d2l.MultiHeadAttention(key_size, query_size, value_size, num_hiddens, num_heads, dropout)
        self.addnorm2 = AddNorm(norm_shape, dropout)

        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm3 = AddNorm(norm_shape, dropout)

    def forward(self, X, state):
        # 编码器的输入由 state 赋予，是 TransformerDecoder 类的成员变量
        enc_output, enc_valid_lens = state[0], state[1]

        # 训练阶段，输出序列的所有词元都在同一时间处理，
        # 因此state[2][self.i]初始化为None。
        # 预测阶段，输出序列是通过词元一个接着一个解码的，
        # 因此state[2][self.i]包含着直到当前时间步第i个块解码的输出表示
        if state[2][self.i] is None:
            key_values = X
        else:
            key_values = torch.cat((state[2][self.i], X), dim=1)
        state[2][self.i] = key_values

        if self.training:
            batch_size, num_step, _ = X.shape
            # dec_valid_lens的开头:(batch_size,num_steps),
            # 其中每一行是[1,2,...,num_steps]
            dec_valid_lens